In [1]:
import os

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    !git clone https://github.com/sambeets/EPE2316-Power-System-Planning.git
    !pip install pypsa
    os.chdir("EPE2316-Power-System-Planning/Assignments/DigiLab Assignment 1")
else:
    print("Running locally, assuming the correct directory is already set.")

Cloning into 'EPE2316-Power-System-Planning'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 374 (delta 11), reused 20 (delta 8), pack-reused 345 (from 1)
Receiving objects: 100% (374/374), 2.74 MiB | 15.41 MiB/s, done.
Resolving deltas: 100% (190/190), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━

# DigiLab Assignment 1
Author: Emil G. Melfald <br>
University of South-Eastern Norway <br>
05.08.2024 <br><br>
This is the first out of two assignments related to the Digital Labs in EPE2316 Power System Planning. The main focus in this assignment is running power flow calculations. The systems you should solve will be slightly more complicated than what is shown in DigiLab 3.

# The assignment system
The following image shows the power system you are to study during the assignment. All variables marked as ? is to be determined by solving the power flow equation. This system builds from DigiLab 3 with two additional lines and on additional bus. In this case, this bus is a PV bus representing a synchronous generator producing 2.0 MW of power.

![Image of the power system under study](https://github.com/sambeets/EPE2316-Power-System-Planning/blob/main/Assignments/DigiLab%20Assignment%201/Assignment_1_Power_System_Drawing.png?raw=1)


## Task 1: Create and simulate in PyPSA
### Task 1.1 - Create the grid
Use the Python module PyPSA to create the network object with all the buses, lines, generators, and loads required according to the figure above.

### Task 1.2 - Simulate the system
Do a power flow of the system. Print out the following for **all buses**:
- Injected active power
- Injected reactive power
- Voltage angle in degrees
- Voltage magnitude in pu

TIP: When defining the voltage setpoint as 1.02 pu at Bus 3, use the keyword argument **v_mag_pu_set=1.02** when defining **Bus 3**.

The results of the power flow should be as following:

Injected active power: [-0.488, -1.5, 2.0] MW <br>
Injected reactive power: [-0.639, -0.5, 1.182] Mvar <br>
Voltages: [1.0, 1.009, 1.02] pu <br>
Voltage angles: [0.0, 0.073, 0.767] degrees <br>

In [2]:
import pypsa
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from DigiLab_supplementary_file import *

network = pypsa.Network()

S_base = 100
V_base = 22


network.add("Bus", "Bus 1", v_nom=V_base, v_mag_pu_set=1.0)     # Slack Bus
network.add("Bus", "Bus 2", v_nom=V_base)                       # PQ Bus
network.add("Bus", "Bus 3", v_nom=V_base, v_mag_pu_set=1.02)    # PV Bus


def pu(R, X, V_base, S_base):
    z_base = (V_base**2)/(S_base*1000) # Z_base in ohms, S_base in MVA
    return R/z_base, X/z_base

R12, X12 = pu(3, 8, V_base, S_base)
R13, X13 = pu(10, 35, V_base, S_base)
R23, X23 = pu(11, 4, V_base, S_base)

network.add("Line", "Line_1-2", bus0="Bus 1", bus1="Bus 2", r=R12, x=X12, s_nom=S_base)
network.add("Line", "Line_1-3", bus0="Bus 1", bus1="Bus 3", r=R13, x=X13, s_nom=S_base)
network.add("Line", "Line_2-3", bus0="Bus 2", bus1="Bus 3", r=R23, x=X23, s_nom=S_base)


network.add("Generator", "Gen1", bus="Bus 1", p_nom=S_base, control="Slack")

network.add("Generator", "Gen3", bus="Bus 3", p_set=2.0, control="PV", v_set=1.02)


network.add("Load", "Load2", bus="Bus 2", p_set=1.5, q_set=0.5)


network.pf()  # Newton-Raphson AC power flow

print("Bus Voltage Magnitude (pu):\n", network.buses_t.v_mag_pu)
print("Bus Voltage Angle (deg):\n", network.buses_t.v_ang * 180 / 3.14159)
print("Generator Active Power Output (MW):\n", network.generators_t.p)
print("Generator Reactive Power Output (Mvar):\n", network.generators_t.q)
print("Line Power Flows (MW):\n", network.lines_t.p0)



Bus Voltage Magnitude (pu):
 name      Bus 1         Bus 2  Bus 3
snapshot                            
now         1.0 -1.396481e+18   1.02
Bus Voltage Angle (deg):
 name      Bus 1          Bus 2          Bus 3
snapshot                                     
now         0.0  152912.977002  153342.994258
Generator Active Power Output (MW):
 name      Gen1  Gen3
snapshot            
now        NaN   2.0
Generator Reactive Power Output (Mvar):
 name              Gen1          Gen3
snapshot                            
now       1.520844e+17  2.850793e+17
Line Power Flows (MW):
 name          Line_1-2  Line_1-3      Line_2-3
snapshot                                      
now      -3.513809e+17  0.019043  3.668031e+35


### End of task 1
---

## Task 2: Set up and solve the system of power flow equations
### Task 2.1 - Create a power flow error function
As was shown in DigiLab 3, a function where the variable vector $y$ was an input, and the power flow error vector $[\Delta P, \Delta Q]$ was returned. Extend the example in DigiLab3 to the system shown above. **Note:** the $Y_{bus}$ matrix is given in the *DigiLab_supplementary_file.py*.

### Task 2.2 - Solve the power flow error equations
Use scipy.optimize.root to solve the power flow error equations to obtain the correct voltage and voltage angles ov the system. Proceed by calculating the injected active and reactive power for all buses. Note that the solution should be exactly the same as obtained from PyPSA. Print out the following for **all buses**:
- Injected active power
- Injected reactive power
- Voltage angle in degrees
- Voltage magnitude in pu

In [12]:
import numpy as np
from scipy.optimize import root
from DigiLab_supplementary_file import Y_bus
S_base = 100  # MVA
V_base = 22   # kV
P2 = 1.5 / S_base   # pu (PQ bus load)
Q2 = 0.5 / S_base   # pu (PQ bus load)
P3 = 2.0 / S_base   # pu (PV bus generator)
bus_names = ["Slack Bus", "PQ Bus", "PV Bus"]

def power_flow(x, V3_set):
    # x[0]: delta_2, x[1]: delta_3, x[2]: V_2
    delta_1 = 0
    V_1 = 1.0
    delta_2, delta_3, V_2 = x
    V = np.array([
        V_1 * np.exp(1j * delta_1),
        V_2 * np.exp(1j * delta_2),
        V3_set * np.exp(1j * delta_3)
    ])

    S = V * np.conj(Y_bus @ V)
    # Bus order: Slack (1), PQ (2), PV (3)
    # Equations:
    eq1 = S[1].real + P2   # P balance at PQ bus
    eq2 = S[1].imag + Q2   # Q balance at PQ bus
    eq3 = S[2].real - P3   # P balance at PV bus
    return [eq1, eq2, eq3]

# Sweep V3 setpoints
V3_setpoints = np.arange(0.98, 1.06, 0.01)
q3_results = []

for V3 in V3_setpoints:
    x0 = [0.0, 0.0, 1.0]  # initial guesses for delta_2, delta_3, V_2
    sol = root(power_flow, x0, args=(V3,))
    if sol.success:
        # Solve for Q at PV bus (Bus 3)
        delta_2, delta_3, V_2 = sol.x
        V_complex = np.array([
            1.0 * np.exp(1j * 0.0),
            V_2 * np.exp(1j * delta_2),
            V3 * np.exp(1j * delta_3)
        ])
        S = V_complex * np.conj(Y_bus @ V_complex)
        Q3 = S[2].imag * S_base  # Convert pu to MVar
        print(f"V3 setpoint: {V3:.2f} pu, Q3 at PV bus: {Q3:.3f} Mvar")
        q3_results.append((V3, Q3))
    else:
        print(f"No solution for V3 = {V3:.2f}")

# Output as table
print("| V3 setpoint (pu) | Q at PV Bus3 (Mvar) |")
for V, Q in q3_results:
    print(f"| {V:.2f}             | {Q:.3f}                |")



V3 setpoint: 0.98 pu, Q3 at PV bus: -0.975 Mvar
V3 setpoint: 0.99 pu, Q3 at PV bus: -0.452 Mvar
V3 setpoint: 1.00 pu, Q3 at PV bus: 0.081 Mvar
V3 setpoint: 1.01 pu, Q3 at PV bus: 0.626 Mvar
V3 setpoint: 1.02 pu, Q3 at PV bus: 1.182 Mvar
V3 setpoint: 1.03 pu, Q3 at PV bus: 1.750 Mvar
V3 setpoint: 1.04 pu, Q3 at PV bus: 2.329 Mvar
V3 setpoint: 1.05 pu, Q3 at PV bus: 2.919 Mvar
V3 setpoint: 1.06 pu, Q3 at PV bus: 3.521 Mvar
| V3 setpoint (pu) | Q at PV Bus3 (Mvar) |
| 0.98             | -0.975                |
| 0.99             | -0.452                |
| 1.00             | 0.081                |
| 1.01             | 0.626                |
| 1.02             | 1.182                |
| 1.03             | 1.750                |
| 1.04             | 2.329                |
| 1.05             | 2.919                |
| 1.06             | 3.521                |


In [14]:
# Check your score for task 2. Give the following function the output from the scipy.optimize.root function
import numpy as np
from scipy.optimize import root
from DigiLab_supplementary_file import Y_bus  # Make sure this is a 3x3 numpy array

S_base = 100  # MVA system base
P2 = 1.5 / S_base   # pu (PQ bus load at Bus 2)
Q2 = 0.5 / S_base   # pu (PQ bus load at Bus 2)
P3 = 2.0 / S_base   # pu (PV bus generator at Bus 3)
V3_set = 1.02       # PV bus voltage setpoint

Y = Y_bus         # Y_bus must return a 3x3 complex numpy array

def power_flow_error(y):
    # y = [delta_2, delta_3, V_2]
    delta_1 = 0
    V_1 = 1.0
    delta_2, delta_3, V_2 = y
    V = np.array([
        V_1 * np.exp(1j * delta_1),
        V_2 * np.exp(1j * delta_2),
        V3_set * np.exp(1j * delta_3)
    ])
    S = V * np.conj(Y @ V)
    # Power balance equations:
    dP2 = S[1].real + P2      # P equation at PQ bus (Bus 2)
    dQ2 = S[1].imag + Q2      # Q equation at PQ bus (Bus 2)
    dP3 = S[2].real - P3      # P equation at PV bus (Bus 3)
    return [dP2, dQ2, dP3]

# Initial guess: [delta_2, delta_3, V_2]
y0 = [0.0, 0.0, 1.0]
sol = root(power_flow_error, y0)

if not sol.success:
    print("Power flow failed to converge:", sol.message)
else:
    delta_2, delta_3, V_2 = sol.x
    V = np.array([
        1.0 * np.exp(1j * 0.0),
        V_2 * np.exp(1j * delta_2),
        V3_set * np.exp(1j * delta_3)
    ])
    S = V * np.conj(Y @ V)
    bus_names = ["Slack Bus", "PQ Bus", "PV Bus"]
    print(f"{'Bus':<11} {'Vm [pu]':<10} {'Va [deg]':<10} {'P_inj [MW]':<13} {'Q_inj [Mvar]':<14}")
    for idx, name in enumerate(bus_names):
        print(f"{name:<11} {np.abs(V[idx]):<10.3f} {np.angle(V[idx], deg=True):<10.2f} "
              f"{S[idx].real * S_base:<13.2f} {S[idx].imag * S_base:<14.2f}")

Bus         Vm [pu]    Va [deg]   P_inj [MW]    Q_inj [Mvar]  
Slack Bus   1.000      0.00       -0.49         -0.64         
PQ Bus      1.009      0.07       -1.50         -0.50         
PV Bus      1.020      0.77       2.00          1.18          


### End of Task 2
---

## Task 3
Use either PyPSA or your own code for the following task. You are to evaluate the reactive power injection from the generator at bus 3 for different voltage setpoints on $V_3$.  

In [ ]:
V3_vals = [0.95, 1.0, 1.05]
Q3_vals = []

# Insert your python code here


In [ ]:
score_task_3(Q3_vals)

### End of Task 3

## End of Assignment 1
---